In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:24pt;}
div.text_cell_render.rendered_html{font-size:20pt;}
div.text_cell_render ul li, div.text_cell_render ol li p, code{font-size:22pt; line-height:30px;}
div.output {font-size:24pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:24pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:24pt;padding:5px;}
table.dataframe{font-size:24px;}
</style>
"""))

# <span style="color:red">ch6_RNN기반의Seq2Seq(스마트번역기)</span>
- Google Neural Machine Translation(GNMT)
- RNN기반의 Sequence to Sequence 방식
- 인코더(입력) / 디코더(모범출력) 연결구조
- 응용분야 : 자연어

# 1. 패키지 및 하이퍼파라미터

In [2]:
import numpy as np
import pandas as pd
from time import time
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical

# 하이퍼파라미터
MY_HIDDEN = 128
MY_EPOCH  = 500

# 2. 학습 데이터

In [10]:
raw = pd.read_csv('data/translate.csv', header=None)
eng_kor = raw.values.tolist() # 데이터프레임을 list로 변환
print(eng_kor[:3])
print('영-한 번역 데이터 갯수 :', len(eng_kor))

[['cold', '감기'], ['come', '오다'], ['cook', '요리']]
영-한 번역 데이터 갯수 : 110


# 3. 영어알파벳과 한글문자 리스트 만들기

In [29]:
e_alpha = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz']
korean = ''.join([data[1] for data in eng_kor])
k_ch = list(set([ch for ch in korean]))
#k_ch.sort()

k_alpha = pd.read_csv('data/korean.csv', header=None)[0].tolist()
k_alpha == k_ch # 순서와 내용이 같은지 여부

False

In [32]:
from collections import Counter
list1 = ['가', '나', '다']
list2 = ['다', '가', '나']
print(list1==list2)
print(Counter(list1)==Counter(list2))

False
True


In [39]:
alpha = e_alpha + k_alpha
print('영어와 한글 알파벳 :', alpha)
alpha_total_size = len(alpha) # 171 (원핫인코딩 사이즈)
print('전체 알파벳 갯수(원핫인코딩 사이즈) :', alpha_total_size)

영어와 한글 알파벳 : ['S', 'E', 'P', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '가', '각', '간', '감', '개', '거', '것', '게', '계', '고', '관', '광', '구', '굴', '규', '그', '금', '기', '깊', '나', '날', '남', '내', '넓', '녀', '노', '놀', '농', '높', '뉴', '늦', '다', '단', '도', '동', '들', '람', '랑', '래', '램', '류', '름', '릎', '리', '많', '망', '매', '머', '먼', '멍', '메', '명', '모', '목', '무', '물', '미', '바', '반', '방', '번', '복', '부', '분', '붕', '비', '뿌', '사', '상', '색', '생', '서', '선', '소', '손', '수', '쉽', '스', '시', '식', '실', '싸', '아', '약', '얇', '어', '언', '얼', '여', '연', '오', '옥', '왼', '요', '용', '우', '운', '움', '위', '유', '은', '을', '음', '의', '이', '익', '인', '읽', '입', '자', '작', '장', '적', '제', '좋', '주', '지', '짜', '쪽', '찾', '책', '출', '칙', '크', '키', '탈', '택', '통', '파', '팔', '편', '피', '핑', '한', '합', '해', '행', '험', '회', '획', '휴', '흐']
전체 알파벳 갯수(원핫인코딩 사이즈) : 171


# 4. 문자당 num를 갖는 dict

In [40]:
char_to_num = {}
for i, c in enumerate(alpha):
    print(i, c)

0 S
1 E
2 P
3 a
4 b
5 c
6 d
7 e
8 f
9 g
10 h
11 i
12 j
13 k
14 l
15 m
16 n
17 o
18 p
19 q
20 r
21 s
22 t
23 u
24 v
25 w
26 x
27 y
28 z
29 가
30 각
31 간
32 감
33 개
34 거
35 것
36 게
37 계
38 고
39 관
40 광
41 구
42 굴
43 규
44 그
45 금
46 기
47 깊
48 나
49 날
50 남
51 내
52 넓
53 녀
54 노
55 놀
56 농
57 높
58 뉴
59 늦
60 다
61 단
62 도
63 동
64 들
65 람
66 랑
67 래
68 램
69 류
70 름
71 릎
72 리
73 많
74 망
75 매
76 머
77 먼
78 멍
79 메
80 명
81 모
82 목
83 무
84 물
85 미
86 바
87 반
88 방
89 번
90 복
91 부
92 분
93 붕
94 비
95 뿌
96 사
97 상
98 색
99 생
100 서
101 선
102 소
103 손
104 수
105 쉽
106 스
107 시
108 식
109 실
110 싸
111 아
112 약
113 얇
114 어
115 언
116 얼
117 여
118 연
119 오
120 옥
121 왼
122 요
123 용
124 우
125 운
126 움
127 위
128 유
129 은
130 을
131 음
132 의
133 이
134 익
135 인
136 읽
137 입
138 자
139 작
140 장
141 적
142 제
143 좋
144 주
145 지
146 짜
147 쪽
148 찾
149 책
150 출
151 칙
152 크
153 키
154 탈
155 택
156 통
157 파
158 팔
159 편
160 피
161 핑
162 한
163 합
164 해
165 행
166 험
167 회
168 획
169 휴
170 흐
